In [32]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [33]:
# load in model
model = joblib.load('new_model.pkl')

In [34]:
df = pd.read_csv('ted_data.csv')

In [35]:
df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Season
0,1.0,Fri,2018-08-10,20:00,Manchester Utd,1.5,2–1,1.8,Leicester City,74439.0,Old Trafford,Andre Marriner,2018-2019
1,1.0,Sat,2018-08-11,12:30,Newcastle Utd,1.0,1–2,2.0,Tottenham,51749.0,St. James' Park,Martin Atkinson,2018-2019
2,1.0,Sat,2018-08-11,15:00,Fulham,0.7,0–2,1.0,Crystal Palace,24821.0,Craven Cottage,Mike Dean,2018-2019
3,1.0,Sat,2018-08-11,15:00,Bournemouth,2.2,2–0,1.4,Cardiff City,10353.0,Vitality Stadium,Kevin Friend,2018-2019
4,1.0,Sat,2018-08-11,15:00,Watford,1.2,2–0,0.3,Brighton,20051.0,Vicarage Road Stadium,Jonathan Moss,2018-2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,38.0,Sun,2025-05-25,16:00,Fulham,NaN,NaN,NaN,Manchester City,NaN,Craven Cottage,NaN,2024-2025
2656,38.0,Sun,2025-05-25,16:00,Nott'ham Forest,NaN,NaN,NaN,Chelsea,NaN,The City Ground,NaN,2024-2025
2657,38.0,Sun,2025-05-25,16:00,Manchester Utd,NaN,NaN,NaN,Aston Villa,NaN,Old Trafford,NaN,2024-2025
2658,38.0,Sun,2025-05-25,16:00,Wolves,NaN,NaN,NaN,Brentford,NaN,Molineux Stadium,NaN,2024-2025


In [36]:
# create a variable called current day
current_day = pd.to_datetime('today').date()

# convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date']).dt.date

# create a df to store our latest data on teams playing before the next round of games
model_df = df[df['Date'] < current_day]

In [37]:
model_df['Attendance'] = model_df['Attendance'].fillna(0) # setting null values to 0 because the NaN are from Covid-19 games with no attendance

model_df['Attendance'] = model_df['Attendance'].astype(int)

/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_37084/3374579801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Attendance'] = model_df['Attendance'].fillna(0) # setting null values to 0 because the NaN are from Covid-19 games with no attendance
/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_37084/3374579801.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Attendance'] = model_df['Attendance'].astype(int)


In [38]:
model_df[['home_goals', 'away_goals']] = model_df['Score'].str.split('–', expand=True).astype(int)

# rename xG and xG.1 columns to home_xG and away_xG
model_df.rename(columns={'xG': 'home_xg', 'xG.1': 'away_xg'}, inplace=True)

/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_37084/838135310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[['home_goals', 'away_goals']] = model_df['Score'].str.split('–', expand=True).astype(int)
/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_37084/838135310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[['home_goals', 'away_goals']] = model_df['Score'].str.split('–', expand=True).astype(int)
/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_3

In [39]:
def get_result(row):
    if row['home_goals'] > row['away_goals']:
        return 'Home Win'
    elif row['home_goals'] < row['away_goals']:
        return 'Away Win'
    else:
        return 'Draw'
    
model_df['Result'] = model_df.apply(get_result, axis=1)

/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_37084/2600445915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Result'] = model_df.apply(get_result, axis=1)


In [40]:
model_df['Day'] = pd.to_datetime(model_df['Date']).dt.day_name()

model_df['Season'] = model_df['Season'].apply(lambda x: x.split('-')[1])

/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_37084/1618590677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Day'] = pd.to_datetime(model_df['Date']).dt.day_name()
/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_37084/1618590677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Season'] = model_df['Season'].apply(lambda x: x.split('-')[1])


In [41]:
model_df = pd.get_dummies(model_df, columns=['Day'])

model_df.reset_index(drop=True, inplace=True)

In [42]:
for x in model_df.Home.unique():
    temp_df = model_df[(model_df['Home'] == x) | (model_df['Away'] == x)]
    temp_df = temp_df.sort_values(['Date'])

    temp_df['goal_value_to_calculate'] = temp_df.apply(lambda y: y['home_goals'] if y['Home'] == x else y['away_goals'], axis=1)
    temp_df['rolling_avg_goals'] = temp_df['goal_value_to_calculate'].rolling(window = 5, closed = 'left', min_periods = 1).mean()

    for index, row in temp_df.iterrows():
        if row['Home'] == x:
            model_df.at[index, 'home_rolling_avg_goals'] = row['rolling_avg_goals']
        else:
            model_df.at[index, 'away_rolling_avg_goals'] = row['rolling_avg_goals']

In [43]:
for x in model_df.Home.unique():
    temp_df = model_df[(model_df['Home'] == x) | (model_df['Away'] == x)]
    temp_df = temp_df.sort_values(['Date'])

    temp_df['xg_value_to_calculate'] = temp_df.apply(lambda y: y['home_xg'] if y['Home'] == x else y['away_xg'], axis=1)
    temp_df['rolling_avg_xG'] = temp_df['xg_value_to_calculate'].rolling(window = 5, closed = 'left', min_periods = 1).mean()

    for index, row in temp_df.iterrows():
        if row['Home'] == x:
            model_df.at[index, 'home_rolling_avg_xG'] = row['rolling_avg_xG']
        else:
            model_df.at[index, 'away_rolling_avg_xG'] = row['rolling_avg_xG']

In [44]:
model_df = model_df.dropna(subset=['home_rolling_avg_goals', 'away_rolling_avg_goals', 'home_rolling_avg_xG', 'away_rolling_avg_xG'])

In [45]:
# create a new column for the central moving average of the home goals
model_df['home_goals_cma'] = model_df['home_goals'].expanding().mean()
model_df['home_xg_cma'] = model_df['home_xg'].expanding().mean()

# away goals
model_df['away_goals_cma'] = model_df['away_goals'].expanding().mean()
model_df['away_xg_cma'] = model_df['away_xg'].expanding().mean()

# create a new column by dividing the home goals by central moving average
model_df['home_goals_div_cma'] = model_df['home_goals'] / model_df['home_goals_cma']
model_df['home_xg_div_cma'] = model_df['home_xg'] / model_df['home_xg_cma']

# away goals
model_df['away_goals_div_cma'] = model_df['away_goals'] / model_df['away_goals_cma']
model_df['away_xg_div_cma'] = model_df['away_xg'] / model_df['away_xg_cma']

In [46]:
# set Wk column to integer
model_df['Wk'] = model_df['Wk'].astype(int)

# create a new column called quarter. divide the season into 4 quarters using the Wk column and assign a value between 1 and 4
model_df['quarter'] = np.where(model_df['Wk'].astype(int) <= 9, 1,
                np.where(model_df['Wk'].astype(int) <= 18, 2,
                np.where(model_df['Wk'].astype(int) <= 27, 3, 4)))

# using home_goals_div_cma, take the mean of the column for each quarter. put into new column called home_goals_seasonality
model_df['home_goals_seasonality'] = model_df.groupby('quarter')['home_goals_div_cma'].transform('mean')
model_df['home_xg_seasonality'] = model_df.groupby('quarter')['home_xg_div_cma'].transform('mean')

# away goals
model_df['away_goals_seasonality'] = model_df.groupby('quarter')['away_goals_div_cma'].transform('mean')
model_df['away_xg_seasonality'] = model_df.groupby('quarter')['away_xg_div_cma'].transform('mean')

# create a new column called home_goals_deseasonalised by dividing home_goals by home_goals_seasonality
model_df['home_goals_deseasonalised'] = model_df['home_goals'] / model_df['home_goals_seasonality']
model_df['home_xg_deseasonalised'] = model_df['home_xg'] / model_df['home_xg_seasonality']

# away goals
model_df['away_goals_deseasonalised'] = model_df['away_goals'] / model_df['away_goals_seasonality']
model_df['away_xg_deseasonalised'] = model_df['away_xg'] / model_df['away_xg_seasonality']

In [47]:
model_df['Season'] = model_df['Season'].astype(int)
# set home goals and away goals to integer
model_df['home_goals'] = model_df['home_goals'].astype(int)
model_df['away_goals'] = model_df['away_goals'].astype(int)

In [48]:
# Ensure 'Result' is categorical or integer-based
model_df['Result'] = model_df['Result'].astype('category')  # or use int depending on your encoding

# Ensure 'home_goals' and 'away_goals' are integers
model_df['home_goals'] = pd.to_numeric(model_df['home_goals'], errors='coerce').fillna(0).astype(int)
model_df['away_goals'] = pd.to_numeric(model_df['away_goals'], errors='coerce').fillna(0).astype(int)

In [49]:
model_df = pd.get_dummies(model_df, columns=['Home', 'Away', 'Venue'])

In [50]:
features = [column for column in model_df.drop(columns=[
    'Date', 'Time', 'home_xg', 'away_xg', 'home_goals', 'Score', 'Referee', 'Attendance',
    'away_goals', 'Result', 'Season', 'quarter', 'home_goals_cma', 'home_xg_cma', 
    'away_goals_cma', 'away_xg_cma', 'home_goals_div_cma', 'home_xg_div_cma', 
    'away_goals_div_cma', 'away_xg_div_cma'])]

# New games

In [51]:
# create a new dataframe with the teams and the features

teams = df['Home'].unique()

teams_df = pd.DataFrame(teams, columns=['Team'])

# assign the latest 'home_rolling_avg_goals', 'away_rolling_avg_goals' etc. to the teams_df
for team in teams:
    home_rolling_avg_goals = model_df[model_df['Home_' + team] == 1]['home_rolling_avg_goals'].iloc[-1]
    away_rolling_avg_goals = model_df[model_df['Away_' + team] == 1]['away_rolling_avg_goals'].iloc[-1]
    home_rolling_avg_xG = model_df[model_df['Home_' + team] == 1]['home_rolling_avg_xG'].iloc[-1]
    away_rolling_avg_xG = model_df[model_df['Away_' + team] == 1]['away_rolling_avg_xG'].iloc[-1]
    home_goals_seasonality = model_df[model_df['Home_' + team] == 1]['home_goals_seasonality'].iloc[-1]
    home_xg_seasonality = model_df[model_df['Home_' + team] == 1]['home_xg_seasonality'].iloc[-1]
    away_goals_seasonality = model_df[model_df['Away_' + team] == 1]['away_goals_seasonality'].iloc[-1]
    away_xg_seasonality = model_df[model_df['Away_' + team] == 1]['away_xg_seasonality'].iloc[-1]
    home_goals_deseasonalised = model_df[model_df['Home_' + team] == 1]['home_goals_deseasonalised'].iloc[-1]
    home_xg_deseasonalised = model_df[model_df['Home_' + team] == 1]['home_xg_deseasonalised'].iloc[-1]
    away_goals_deseasonalised = model_df[model_df['Away_' + team] == 1]['away_goals_deseasonalised'].iloc[-1]
    away_xg_deseasonalised = model_df[model_df['Away_' + team] == 1]['away_xg_deseasonalised'].iloc[-1]

    teams_df.loc[teams_df['Team'] == team, 'home_rolling_avg_goals'] = home_rolling_avg_goals
    teams_df.loc[teams_df['Team'] == team, 'away_rolling_avg_goals'] = away_rolling_avg_goals
    teams_df.loc[teams_df['Team'] == team, 'home_rolling_avg_xG'] = home_rolling_avg_xG
    teams_df.loc[teams_df['Team'] == team, 'away_rolling_avg_xG'] = away_rolling_avg_xG
    teams_df.loc[teams_df['Team'] == team, 'home_goals_seasonality'] = home_goals_seasonality
    teams_df.loc[teams_df['Team'] == team, 'home_xg_seasonality'] = home_xg_seasonality
    teams_df.loc[teams_df['Team'] == team, 'away_goals_seasonality'] = away_goals_seasonality
    teams_df.loc[teams_df['Team'] == team, 'away_xg_seasonality'] = away_xg_seasonality
    teams_df.loc[teams_df['Team'] == team, 'home_goals_deseasonalised'] = home_goals_deseasonalised
    teams_df.loc[teams_df['Team'] == team, 'home_xg_deseasonalised'] = home_xg_deseasonalised
    teams_df.loc[teams_df['Team'] == team, 'away_goals_deseasonalised'] = away_goals_deseasonalised
    teams_df.loc[teams_df['Team'] == team, 'away_xg_deseasonalised'] = away_xg_deseasonalised


# create a copy of original df and filter for games this weekend
weekend_df = df.copy()

# filter for Wk 9, season 2024-2025
weekend_df = weekend_df[(weekend_df['Wk'] == 9) & (weekend_df['Season'] == '2024-2025')]

# drop columns with null values
weekend_df = weekend_df.drop(columns=['xG', 'xG.1', 'Attendance', 'Referee', 'Score'])


# using Home and Away columns, assign the rolling averages and seasonality values to the weekend_df
for index, row in weekend_df.iterrows():
    home_team = row['Home']
    away_team = row['Away']

    home_rolling_avg_goals = teams_df[teams_df['Team'] == home_team]['home_rolling_avg_goals'].iloc[0]
    away_rolling_avg_goals = teams_df[teams_df['Team'] == away_team]['away_rolling_avg_goals'].iloc[0]
    home_rolling_avg_xG = teams_df[teams_df['Team'] == home_team]['home_rolling_avg_xG'].iloc[0]
    away_rolling_avg_xG = teams_df[teams_df['Team'] == away_team]['away_rolling_avg_xG'].iloc[0]
    home_goals_seasonality = teams_df[teams_df['Team'] == home_team]['home_goals_seasonality'].iloc[0]
    home_xg_seasonality = teams_df[teams_df['Team'] == home_team]['home_xg_seasonality'].iloc[0]
    away_goals_seasonality = teams_df[teams_df['Team'] == away_team]['away_goals_seasonality'].iloc[0]
    away_xg_seasonality = teams_df[teams_df['Team'] == away_team]['away_xg_seasonality'].iloc[0]
    home_goals_deseasonalised = teams_df[teams_df['Team'] == home_team]['home_goals_deseasonalised'].iloc[0]
    home_xg_deseasonalised = teams_df[teams_df['Team'] == home_team]['home_xg_deseasonalised'].iloc[0]
    away_goals_deseasonalised = teams_df[teams_df['Team'] == away_team]['away_goals_deseasonalised'].iloc[0]
    away_xg_deseasonalised = teams_df[teams_df['Team'] == away_team]['away_xg_deseasonalised'].iloc[0]

    weekend_df.at[index, 'home_rolling_avg_goals'] = home_rolling_avg_goals
    weekend_df.at[index, 'away_rolling_avg_goals'] = away_rolling_avg_goals
    weekend_df.at[index, 'home_rolling_avg_xG'] = home_rolling_avg_xG
    weekend_df.at[index, 'away_rolling_avg_xG'] = away_rolling_avg_xG
    weekend_df.at[index, 'home_goals_seasonality'] = home_goals_seasonality
    weekend_df.at[index, 'home_xg_seasonality'] = home_xg_seasonality

    weekend_df.at[index, 'away_goals_seasonality'] = away_goals_seasonality
    weekend_df.at[index, 'away_xg_seasonality'] = away_xg_seasonality
    weekend_df.at[index, 'home_goals_deseasonalised'] = home_goals_deseasonalised
    weekend_df.at[index, 'home_xg_deseasonalised'] = home_xg_deseasonalised
    weekend_df.at[index, 'away_goals_deseasonalised'] = away_goals_deseasonalised
    weekend_df.at[index, 'away_xg_deseasonalised'] = away_xg_deseasonalised


display(weekend_df)

,Wk,Day,Date,Time,Home,Away,Venue,Season,home_rolling_avg_goals,away_rolling_avg_goals,home_rolling_avg_xG,away_rolling_avg_xG,home_goals_seasonality,home_xg_seasonality,away_goals_seasonality,away_xg_seasonality,home_goals_deseasonalised,home_xg_deseasonalised,away_goals_deseasonalised,away_xg_deseasonalised
2360,9.0,Fri,2024-10-25,20:00,Leicester City,Nott'ham Forest,King Power Stadium,2024-2025,1.4,1.0,0.64,1.16,1.012575,1.01772,1.047358,1.030433,0.987581,0.786071,0.954784,0.873419
2361,9.0,Sat,2024-10-26,15:00,Brentford,Ipswich Town,Gtech Community Stadium,2024-2025,1.2,1.0,1.10,0.76,1.012575,1.01772,1.047358,1.030433,4.937906,4.126871,0.954784,0.582279
2362,9.0,Sat,2024-10-26,15:00,Brighton,Wolves,The American Express Stadium,2024-2025,1.4,1.2,1.50,0.98,1.012575,1.01772,1.047358,1.030433,2.962743,1.768659,2.864351,0.970465
2363,9.0,Sat,2024-10-26,15:00,Manchester City,Southampton,Etihad Stadium,2024-2025,2.4,0.6,2.28,1.14,1.012575,1.01772,1.047358,1.030433,2.962743,1.572141,0.954784,0.582279
2364,9.0,Sat,2024-10-26,15:00,Aston Villa,Bournemouth,Villa Park,2024-2025,2.0,1.4,1.44,1.74,1.012575,1.01772,1.047358,1.030433,0.000000,0.491294,0.000000,2.037977
2365,9.0,Sat,2024-10-26,17:30,Everton,Fulham,Goodison Park,2024-2025,1.4,1.6,1.14,1.54,1.012575,1.01772,1.047358,1.030433,0.000000,0.687812,1.909567,2.523210
2366,9.0,Sun,2024-10-27,14:00,Chelsea,Newcastle Utd,Stamford Bridge,2024-2025,3.0,1.4,2.24,1.60,1.012575,1.01772,1.047358,1.030433,0.987581,2.259953,0.000000,2.037977
2367,9.0,Sun,2024-10-27,14:00,West Ham,Manchester Utd,London Stadium,2024-2025,1.0,0.8,0.96,1.60,1.012575,1.01772,1.047358,1.030433,3.950324,3.537318,0.000000,0.582279
2368,9.0,Sun,2024-10-27,14:00,Crystal Palace,Tottenham,Selhurst Park,2024-2025,0.8,2.2,1.24,2.44,1.012575,1.01772,1.047358,1.030433,0.000000,0.589553,1.909567,1.261605
2369,9.0,Sun,2024-10-27,16:30,Arsenal,Liverpool,Emirates Stadium,2024-2025,2.0,2.0,1.76,1.94,1.012575,1.01772,1.047358,1.030433,2.962743,2.751247,0.954784,1.358652


In [52]:
teams_df

,Team,home_rolling_avg_goals,away_rolling_avg_goals,home_rolling_avg_xG,away_rolling_avg_xG,home_goals_seasonality,home_xg_seasonality,away_goals_seasonality,away_xg_seasonality,home_goals_deseasonalised,home_xg_deseasonalised,away_goals_deseasonalised,away_xg_deseasonalised
0,Manchester Utd,0.6,0.8,1.44,1.60,1.012575,1.017720,1.047358,1.030433,1.975162,1.277365,0.000000,0.582279
1,Newcastle Utd,1.2,1.4,1.70,1.60,1.012575,1.017720,1.047358,1.030433,0.000000,1.965177,0.000000,2.037977
2,Fulham,1.6,1.6,1.70,1.54,1.012575,1.017720,1.047358,1.030433,0.987581,1.768659,1.909567,2.523210
3,Bournemouth,1.2,1.4,1.72,1.74,1.012575,1.017720,1.047358,1.030433,1.975162,1.768659,0.000000,2.037977
4,Watford,0.6,0.6,0.78,0.72,1.075523,1.076814,1.051512,1.036789,0.929781,1.300132,0.951012,0.578710
5,Huddersfield,0.4,0.6,0.96,0.94,1.075523,1.076814,1.051512,1.036789,0.929781,0.928666,0.951012,0.675161
6,Wolves,1.4,1.2,0.80,0.98,1.012575,1.017720,1.047358,1.030433,0.987581,0.786071,2.864351,0.970465
7,Southampton,0.8,0.6,1.24,1.14,1.012575,1.017720,1.047358,1.030433,1.975162,2.063435,0.954784,0.582279
8,Liverpool,1.8,2.0,1.72,1.94,1.012575,1.017720,1.047358,1.030433,1.975162,1.866918,0.954784,1.358652
9,Arsenal,2.0,2.2,1.76,2.14,1.012575,1.017720,1.047358,1.030433,2.962743,2.751247,0.000000,0.679326


In [53]:
weekend_df['Day'] = pd.to_datetime(weekend_df['Date']).dt.day_name()

# store weekend_df in new variable for final results

results_df = weekend_df.loc[:, ['Date', 'Home', 'Away', 'Day', 'Venue']]

# Ensure 'weekend_df' has the same structure as 'model_df'
weekend_df = pd.get_dummies(weekend_df, columns=['Home', 'Away', 'Day','Venue'], drop_first=False)

# Add missing columns with default values
for column in model_df.columns:
    if column not in weekend_df.columns:
        if model_df[column].dtype == 'bool':
            weekend_df[column] = False
        elif model_df[column].dtype == 'float64':
            weekend_df[column] = 0.0
        elif model_df[column].dtype == 'int64':
            weekend_df[column] = 0
        else:
            weekend_df[column] = None

# Ensure the order of columns matches
weekend_df = weekend_df[model_df.columns]

/var/folders/ky/0j3p160j7vl9mg1glj79h0q40000gn/T/ipykernel_37084/27419728.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weekend_df[column] = False


In [54]:
weekend_df

,Wk,Date,Time,home_xg,Score,away_xg,Attendance,Referee,Season,home_goals,...,Venue_The American Express Stadium,Venue_The City Ground,Venue_The Hawthorns,Venue_The John Smith's Stadium,Venue_Tottenham Hotspur Stadium,Venue_Turf Moor,Venue_Vicarage Road Stadium,Venue_Villa Park,Venue_Vitality Stadium,Venue_Wembley Stadium
2360,9.0,2024-10-25,20:00,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,False,False,False
2361,9.0,2024-10-26,15:00,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,False,False,False
2362,9.0,2024-10-26,15:00,0.0,None,0.0,0,None,2024-2025,0,...,True,False,False,False,False,False,False,False,False,False
2363,9.0,2024-10-26,15:00,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,False,False,False
2364,9.0,2024-10-26,15:00,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,True,False,False
2365,9.0,2024-10-26,17:30,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,False,False,False
2366,9.0,2024-10-27,14:00,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,False,False,False
2367,9.0,2024-10-27,14:00,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,False,False,False
2368,9.0,2024-10-27,14:00,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,False,False,False
2369,9.0,2024-10-27,16:30,0.0,None,0.0,0,None,2024-2025,0,...,False,False,False,False,False,False,False,False,False,False


In [55]:
features = [column for column in weekend_df.drop(columns=[
    'Date', 'Time', 'home_xg', 'away_xg', 'home_goals', 'Score', 'Attendance', 'Referee',
    'away_goals', 'Result', 'Season', 'quarter', 'home_goals_cma', 'home_xg_cma', 
    'away_goals_cma', 'away_xg_cma', 'home_goals_div_cma', 'home_xg_div_cma', 
    'away_goals_div_cma', 'away_xg_div_cma'])]

In [56]:
# add the weekend_df Result column to the results_df
results_df['results'] = model.predict(weekend_df[features])


In [59]:
results_df.to_csv('prediction_wk9_2025.csv', index=False)

In [58]:
features

['Wk',
 'Day_Friday',
 'Day_Monday',
 'Day_Saturday',
 'Day_Sunday',
 'Day_Thursday',
 'Day_Tuesday',
 'Day_Wednesday',
 'home_rolling_avg_goals',
 'away_rolling_avg_goals',
 'home_rolling_avg_xG',
 'away_rolling_avg_xG',
 'home_goals_seasonality',
 'home_xg_seasonality',
 'away_goals_seasonality',
 'away_xg_seasonality',
 'home_goals_deseasonalised',
 'home_xg_deseasonalised',
 'away_goals_deseasonalised',
 'away_xg_deseasonalised',
 'Home_Arsenal',
 'Home_Aston Villa',
 'Home_Bournemouth',
 'Home_Brentford',
 'Home_Brighton',
 'Home_Burnley',
 'Home_Cardiff City',
 'Home_Chelsea',
 'Home_Crystal Palace',
 'Home_Everton',
 'Home_Fulham',
 'Home_Huddersfield',
 'Home_Ipswich Town',
 'Home_Leeds United',
 'Home_Leicester City',
 'Home_Liverpool',
 'Home_Luton Town',
 'Home_Manchester City',
 'Home_Manchester Utd',
 'Home_Newcastle Utd',
 'Home_Norwich City',
 "Home_Nott'ham Forest",
 'Home_Sheffield Utd',
 'Home_Southampton',
 'Home_Tottenham',
 'Home_Watford',
 'Home_West Brom',
 'Home